In [1]:
import pandas as pd
import numpy as np
import os
import csv
from tqdm import tqdm, trange
import random

# Scikit-learn
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier # RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, roc_auc_score # mean_squared error, roc_auc_score, accuracy_score, f1 score
from sklearn.linear_model import LogisticRegression
#

import warnings
warnings.filterwarnings('ignore')

In [2]:
dataframes = []
#init_data = pd.read_csv(f'/Users/chrisjeff/Desktop/Origin/Cleaned_data/cleaned_data_199611020ATL.csv')
#init_data['Win'] = 0

#dataframes.append(init_data)
total_wanted_files = 2000 #1000 files to be used
files_used = []

for file_name in tqdm(os.listdir('Cleaned_data'), desc='Read Files'):
  if total_wanted_files == 0:
    break
  #if file_name == 'cleaned_data_199611020ATL.csv': #skip initial file
    #continue
  _data = pd.read_csv(f'./Cleaned_data/{file_name}')
  _data['Win'] = 0
  files_used.append(file_name)
  total_wanted_files -= 1

  

  dataframes.append(_data)

data = pd.concat(dataframes)
print(files_used)
print(data.columns)

Read Files:   6%|▌         | 2000/32881 [00:29<07:32, 68.24it/s] 


['cleaned_data_200012080CHH.csv', 'cleaned_data_201903100MIN.csv', 'cleaned_data_199711130DAL.csv', 'cleaned_data_200305220NJN.csv', 'cleaned_data_201901160LAC.csv', 'cleaned_data_201212260UTA.csv', 'cleaned_data_201703250POR.csv', 'cleaned_data_200301150PHI.csv', 'cleaned_data_201112250DAL.csv', 'cleaned_data_200201230TOR.csv', 'cleaned_data_200812120PHO.csv', 'cleaned_data_201703010ORL.csv', 'cleaned_data_201802110HOU.csv', 'cleaned_data_199902270CHI.csv', 'cleaned_data_199905120NYK.csv', 'cleaned_data_199611130NJN.csv', 'cleaned_data_199903250ORL.csv', 'cleaned_data_199801140ATL.csv', 'cleaned_data_200604170DEN.csv', 'cleaned_data_200712210POR.csv', 'cleaned_data_201212100PHI.csv', 'cleaned_data_199702050LAL.csv', 'cleaned_data_200705070DET.csv', 'cleaned_data_200611060CHI.csv', 'cleaned_data_200803190IND.csv', 'cleaned_data_201902210LAL.csv', 'cleaned_data_202204230TOR.csv', 'cleaned_data_202304150PHI.csv', 'cleaned_data_200205040SAC.csv', 'cleaned_data_200111060MIN.csv', 'cleaned_

Functions that converts information

In [3]:
def convert_pos(s):
  if len(s) == 1:
    if s == 'G':
      return 1
    elif s == 'F':
      return 3
    elif s == 'C':
      return 5
    
  elif len(s) == 2:
    if s == 'PG':
      return 1
    elif s == 'SG' or s == 'GF':
      return 2
    elif s == 'SF':
      return 3
    elif s == 'PF' or s == 'FC':
      return 4
    
  elif len(s) == 3:
    if s == 'G-F' or s == 'F-G':
      return 2
    elif s == 'F-C' or s == 'C-F':
      return 4


def convert_term(term):
  if term == 'Q1':
    return 1
  elif term == 'Q2':
    return 2
  elif term == 'H2':
    return 3
  elif term == 'Q3':
    return 4
  elif term == 'Q4':
    return 5
  elif term == 'H2':
    return 6

Insert position for every player

In [4]:
temp = data.copy(deep=True)


#for row_idx in range(len(temp)):
  #print(temp.loc[row_idx, :])

#remove null rows, percentages
temp = temp.drop(['FG%', '3P%', 'FT%', 'Name'], axis=1)
temp.dropna(axis=0, inplace=True)
display(temp)

,Home Team,Term,Time Played (sec),FG,FGA,3P,3PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Win
0,0,Q1,720.0,3.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,6.0,0.0,0
1,0,Q2,409.0,2.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,5.0,0.0,0
2,0,H1,1129.0,5.0,8.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,11.0,0.0,0
3,0,Q3,431.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,4.0,-5.0,0
4,0,Q4,720.0,1.0,3.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,6.0,-2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,1,H1,219.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-4.0,0
124,1,Q4,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0
125,1,H2,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0
129,1,Q3,77.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0


Find the winning team and identify the players on that team

In [5]:
for file_name in tqdm(files_used, desc='Files Adjusted'):
  _data = pd.read_csv(f'./Cleaned_data/{file_name}').copy(deep=True)
  _data['Win'] = 0
  _temp = data.loc[:, 'Home Team' : ]
  away_total_pts = 0
  home_total_pts = 0

  away_players = []
  home_players = []

  for i in range(len(_data)):
    #if (i + 1) % 3 == 0:
      #continue
    try:
      if temp.iloc[i,0] == 0:
        away_total_pts += int(temp.iloc[i,-3])
        away_players.append(i)
        #print(f'total points so far : {away_total_pts}')
      else:
        home_total_pts += int(temp.iloc[i,-3])
        home_players.append(i)
      #print(data.iloc[i,1])
    except:
      continue
  #print(f'away points total : {away_total_pts}')
  #print(f'home points total : {home_total_pts}')
  
  if away_total_pts > home_total_pts:
    for idx in away_players:
      temp.iloc[idx, -1] = 1
    #temp.loc[away_total_pts > home_total_pts, 'Win'] = 1
    #players_won = temp[temp['Home Team'] == 0]
    #print(players_won)
    #players_won['Win'] = 1
  elif home_total_pts > away_total_pts:
    for idx in home_players:
      temp.iloc[idx, -1] = 1
    #temp.loc[home_total_pts > away_total_pts, 'Win'] = 1
    #players_won = temp[temp['Home Team'] == 1]
    #print(players_won)
    #players_won['Win'] = 1
print(temp)

Files Adjusted: 100%|██████████| 2000/2000 [02:38<00:00, 12.63it/s]


     Home Team Term  Time Played (sec)   FG  FGA   3P  3PA   FT  FTA  ORB  \
0            0   Q1              720.0  3.0  6.0  0.0  0.0  0.0  0.0  1.0   
1            0   Q2              409.0  2.0  2.0  0.0  0.0  1.0  2.0  0.0   
2            0   H1             1129.0  5.0  8.0  0.0  0.0  1.0  2.0  1.0   
3            0   Q3              431.0  2.0  3.0  0.0  0.0  0.0  0.0  0.0   
4            0   Q4              720.0  1.0  3.0  0.0  1.0  4.0  4.0  0.0   
..         ...  ...                ...  ...  ...  ...  ...  ...  ...  ...   
122          1   H1              219.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0   
124          1   Q4               35.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
125          1   H2               35.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
129          1   Q3               77.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   
131          1   H2               77.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   

     DRB  TRB  AST  STL  BLK  TOV   PF   PTS  +/-  Win  
0    0.0  1.0  1.0

Convert terms into usuable data

In [6]:
for i in trange(len(temp), desc='Term adjusted rows'):
  temp.iloc[i, 1] = convert_term(temp.iloc[i, 1])
  #if pd.isna(temp.iloc[i, 3]):
    #temp.drop(i)

print(temp)

Term adjusted rows:   0%|          | 0/215170 [00:00<?, ?it/s]

Term adjusted rows: 100%|██████████| 215170/215170 [11:39<00:00, 307.55it/s]

     Home Team  Term  Time Played (sec)   FG  FGA   3P  3PA   FT  FTA  ORB  \
0            0     1              720.0  3.0  6.0  0.0  0.0  0.0  0.0  1.0   
1            0     2              409.0  2.0  2.0  0.0  0.0  1.0  2.0  0.0   
2            0  None             1129.0  5.0  8.0  0.0  0.0  1.0  2.0  1.0   
3            0     4              431.0  2.0  3.0  0.0  0.0  0.0  0.0  0.0   
4            0     5              720.0  1.0  3.0  0.0  1.0  4.0  4.0  0.0   
..         ...   ...                ...  ...  ...  ...  ...  ...  ...  ...   
122          1  None              219.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0   
124          1     5               35.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
125          1     3               35.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
129          1     4               77.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   
131          1     3               77.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   

     DRB  TRB  AST  STL  BLK  TOV   PF   PTS  +/-  Win  
0    0

In [14]:
#print(temp.columns)
#temp.drop('PTS', axis=1, inplace=True)
temp.dropna(inplace=True)


rand_data = temp.copy(deep=True)
num_rand_features = 10
all_features = len(rand_data.columns) - 1
rand_indices = random.sample(range(all_features), num_rand_features)


rand_features = rand_data.iloc[:, rand_indices]
print(rand_features)


rand_arr = rand_features.to_numpy()
arr = temp.to_numpy()
x = rand_arr[:, :]
y = arr[:, -1:].astype('int')

     3PA   PF  ORB   FG  AST  Home Team  DRB   FT Term  BLK
0    0.0  0.0  1.0  3.0  1.0          0  0.0  0.0    1  0.0
1    0.0  1.0  0.0  2.0  2.0          0  0.0  1.0    2  0.0
3    0.0  0.0  0.0  2.0  3.0          0  0.0  0.0    4  0.0
4    1.0  1.0  0.0  1.0  1.0          0  0.0  4.0    5  0.0
5    1.0  1.0  0.0  3.0  4.0          0  0.0  4.0    3  0.0
..   ...  ...  ...  ...  ...        ...  ...  ...  ...  ...
121  1.0  0.0  0.0  0.0  1.0          1  0.0  0.0    2  0.0
124  0.0  0.0  0.0  0.0  0.0          1  0.0  0.0    5  0.0
125  0.0  0.0  0.0  0.0  0.0          1  0.0  0.0    3  0.0
129  0.0  0.0  0.0  0.0  0.0          1  0.0  0.0    4  0.0
131  0.0  0.0  0.0  0.0  0.0          1  0.0  0.0    3  0.0

[177056 rows x 10 columns]


In [15]:
print(temp.isna().sum())
y = y.ravel()

Home Team            0
Term                 0
Time Played (sec)    0
FG                   0
FGA                  0
3P                   0
3PA                  0
FT                   0
FTA                  0
ORB                  0
DRB                  0
TRB                  0
AST                  0
STL                  0
BLK                  0
TOV                  0
PF                   0
+/-                  0
Win                  0
dtype: int64


In [16]:
cv_inner = KFold(n_splits=2, shuffle=True, random_state=42)          #1
cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
# 10x2 nested cross validation
# 9 folds for training, 1 fold for testing
# 9 folds --> 50% (training) - 50% (validation) split 
regressor = GradientBoostingClassifier()       # RandomForestClassifier, GradientBoostingClassifier, LogisticRegression        #2

p_grid = {#'bootstrap': [True, False],
        'max_depth': [20, 40, 60, None],
        'max_features': [None, 'sqrt'],
        'min_samples_leaf': [1, 2, 4],
        'min_samples_split': [2, 5, 10],
        'n_estimators': [200, 1000, 1600]}


#Gradient Boosting Classifier, roc_auc
#p_grid = {'max_depth': [40, 80, 120, 160, None],
#        'max_features': [None, 'sqrt'],
#        'min_samples_leaf': [1, 2, 4, 6, 8],
#        'min_samples_split': [1, 2, 5, 8, 10],
#        'n_estimators': [500, 1000, 1500, 1800, 2000]}

history = []
pointer = 1
for train_index, test_index in tqdm(cv_outer.split(x, y), desc='Times Looped'):                          #3
    print('\nNestedCV: {} of outer fold {}'.format(pointer, cv_outer.get_n_splits()))
    x_train, x_test = x[train_index], x[test_index] # x (features)
    y_train, y_test = y[train_index], y[test_index] # y (target)

    model = GridSearchCV(regressor, param_grid=p_grid,
                        scoring='roc_auc', cv=cv_inner, n_jobs=-1) # classification --> roc_auc, f1, accuracy   # Grid search  #4
    model.fit(x_train, y_train) # trains ML model
    
    pred_test = model.predict(x_test) # predict test data
    pred_training = model.predict(x_train) # predict training data
    
    # Evaluate model performance
    # scikit learn accuracy
    auc_train = roc_auc_score(y_train, pred_training)     #roc_auc_score, f1, accuracy       #5
    auc_test = roc_auc_score(y_test, pred_test)       #roc_auc_score
    
    print("""
    Best set of parameters: {}
    Best MSE              : {:.2f}

    Training
        MSE: {:.3f}
    Test
        MSE: {:.3f}
    """.format(
        model.best_params_,
        model.best_score_,
        auc_train,
        auc_test,
        )
    )
    history.append(auc_test)
    pointer += 1

print('Overall test performance: {:.2f}'.format(np.mean(history)))

Times Looped: 0it [00:00, ?it/s]


NestedCV: 1 of outer fold 10
